In [7]:
!pip install keras-preprocessing



In [40]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [41]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [42]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

In [48]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)
print(train)

surprise completed
fear completed
.ipynb_checkpoints completed
neutral completed
angry completed
happy completed
sad completed
                               image     label
0     images/train/surprise/3017.jpg  surprise
1     images/train/surprise/2023.jpg  surprise
2      images/train/surprise/492.jpg  surprise
3     images/train/surprise/3060.jpg  surprise
4     images/train/surprise/1528.jpg  surprise
...                              ...       ...
1990        images/train/sad/590.jpg       sad
1991       images/train/sad/3082.jpg       sad
1992       images/train/sad/1559.jpg       sad
1993         images/train/sad/72.jpg       sad
1994        images/train/sad/957.jpg       sad

[1995 rows x 2 columns]


In [49]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)
print(test)

surprise completed
fear completed
.ipynb_checkpoints completed
neutral completed
angry completed
happy completed
sad completed
                              image     label
0    images/test/surprise/16829.jpg  surprise
1     images/test/surprise/1249.jpg  surprise
2    images/test/surprise/15486.jpg  surprise
3     images/test/surprise/3223.jpg  surprise
4     images/test/surprise/3002.jpg  surprise
..                              ...       ...
625        images/test/sad/4274.jpg       sad
626        images/test/sad/1678.jpg       sad
627        images/test/sad/2633.jpg       sad
628        images/test/sad/2499.jpg       sad
629        images/test/sad/1931.jpg       sad

[630 rows x 2 columns]


In [50]:
from tqdm.notebook import tqdm

# Data Preprocessing

In [51]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        try:
            # Load the image
            img = load_img(image, color_mode='grayscale')
            # Convert to numpy array
            img = np.array(img)
            features.append(img)
        except Exception as e:
            print(f"Error processing {image}: {e}")
    return features

# Feature Extraction

In [52]:
train_features = extract_features(train['image'])
test_features = extract_features(test['image'])

  0%|          | 0/1995 [00:00<?, ?it/s]

  0%|          | 0/630 [00:00<?, ?it/s]

Error processing images/test/surprise/.ipynb_checkpoints: [Errno 21] Is a directory: 'images/test/surprise/.ipynb_checkpoints'


# Normalization

In [53]:
'''
x_train = train_features / 255.0
x_test = test_features / 255.0
'''

'\nx_train = train_features / 255.0\nx_test = test_features / 255.0\n'

In [80]:
import numpy as np

# Convert lists to NumPy arrays
train_features = np.array(train_features)
test_features = np.array(test_features)

# Perform element-wise division
x_train = train_features / 255.0
x_test = test_features / 255.0


In [81]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['label'])
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [82]:
print(len(x_train))  # Should print 629
print(len(y_train))  # Should print 629


1995
1995


In [83]:
if len(x_train) > len(y_train):
    x_train = x_train[:len(y_train)]
elif len(y_train) > len(x_train):
    y_train = y_train[:len(x_train)]


In [84]:
'''
# If x_train has fewer samples, you can truncate y_train
x_train = x_train[:len(y_train)]

# If y_train has fewer samples, you can truncate x_train
y_train = y_train[:len(x_train)]
'''

'\n# If x_train has fewer samples, you can truncate y_train\nx_train = x_train[:len(y_train)]\n\n# If y_train has fewer samples, you can truncate x_train\ny_train = y_train[:len(x_train)]\n'

In [85]:
from sklearn.utils import resample

# Repeat samples to match size
x_train_resampled, y_train_resampled = resample(x_train, y_train, replace=True, n_samples=len(x_train), random_state=42)


In [86]:
assert len(x_train) == len(y_train), "Mismatch between input and label data"


# Building the Neural Network Model

In [87]:
# sequential model
model = Sequential()

# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# flattening
model.add(Flatten())

# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# output layer
model.add(Dense(7, activation='softmax'))

'''
# model compilation
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )
'''
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


/opt/conda/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Training the Model

In [88]:
model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1901 - loss: 1.8874  

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 629
'y' sizes: 630


# Saving the Model

In [13]:
model_json = model.to_json()
with open("emotiondetector.json", 'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

C:\Users\Star\Anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
